# Lab 2: Airport Graph
In this lab we're implementing a simple, weighted, not directed graph that modelates the traffic routes of a certain number of Airports in the world


## Members
This group is made up of:

- Juan Andrés Povea Fernández
- Yovany Zhu Ye
- Carlos Elías López Gallardo
- Jesús David Cantillo Guerrero


### Importing the necessary libraries

In [ ]:
# Built-in 
from typing import Any, List, Optional, Tuple
from pprint import pprint   
from queue import Queue
from math import pi, sin, cos, atan2, sqrt, radians # We import just what we need
import folium
from IPython.display import HTML, display
from folium import PolyLine


# Third-party
import pandas as pd


### Reading the data

In [ ]:
df = pd.read_csv("flights_final.csv")
df = df.drop_duplicates()

df['Combination'] = df.apply(lambda row: ''.
join(sorted([row['Source Airport Code'], 
row['Destination Airport Code']])), 
axis=1)

symmetric_flights_counter = df.groupby('Combination').size().reset_index(name='Amount symmetric flights')
symmetric_combinations = symmetric_flights_counter[symmetric_flights_counter['Amount symmetric flights'] > 1]

df = df.sort_values(['Source Airport Code', 'Destination Airport Code'])
df = df.drop_duplicates(subset='Combination', keep='first').drop(columns='Combination')


### Haversian formula

In [ ]:
# In this block of code we're declaring a function to calculate the distance between two coordinates
def haversine(latitudeA, longitudeA, latitudeB, longitudeB) ->float:
    earth_radius = 6371.0
    radian_factor = pi/180

    latitudeA *= radian_factor
    latitudeB *= radian_factor

    longitudeA *= radian_factor
    longitudeB *= radian_factor

    a = sin( ((latitudeB-latitudeA)/2) )**2 + cos(latitudeA) * cos(latitudeB) * \
    sin( ((longitudeB-longitudeA)/2) )**2

    c = 2 * atan2( sqrt(a), sqrt(1-a) )  

    return round(earth_radius * c, 1) # Kilometers


### Weighted Graph

In [ ]:
class AirportGraph:

    INF = (1 << 31) - 1

    def __init__(self):
        self.adj_list = {}

    def add_vertex(self, airport_code, 
        airport_name, 
        airport_city, 
        airport_country, 
        latitude, 
        longitude):

        # Add an airport to the adjacency list
        self.adj_list[airport_code] = {
            'airport_code': airport_code,
            'name': airport_name,
            'city': airport_city,
            'country': airport_country,
            'latitude': latitude,
            'longitude': longitude,
            'flights': []
        }

    def add_edge(self, source_code: str, destinatin_code: str, weight: int):
        codes_in_list = (source_code in self.adj_list) and (destination_code in self.adj_list)

        if codes_in_list:
            self.adj_list[source_code]['flights'].append([destination_code, weight])
            self.adj_list[destinatin_code]['flights'].append([source_code, weight])
        else:
            print(f'The codes {source_code} and {destination_code} arent in list...')    

    def dijkstra(self, source_code: str):

        # Variable initialization
        distance = {}
        visited = {}
        pred = {}

        for code in self.adj_list:
          distance[code] = self.INF
          visited[code] = False
          pred[code] = None    

        distance[source_code] = 0 # The distance from the same vertex is zero
        
        while not self.all_visited(visited):
            # Find the airport with the smallest distance among unvisited airports
            min_distance_airport = self.min_dist_not_visit(distance, visited)

            # Mark the selected airport as visited
            visited[min_distance_airport] = True

            # Update distances for neighboring airports
            for flight in self.adj_list[min_distance_airport]['flights']:
                destination = flight[0]
                current_distance = flight[1]
                if current_distance !=0 and distance[min_distance_airport] + current_distance < distance[destination] and not visited[destination]:
                    distance[destination] = round(distance[min_distance_airport] + current_distance, 1)
                    pred[destination] = min_distance_airport

        return distance, pred           


    # This method obtains the 10 largest paths from a given airport!
    def largest_paths(self, airport_source_code: str):

        # We execute the dijsktra algorithm to find the shortest paths from the source airport
        paths, pred  = self.dijkstra(airport_source_code)

        # We want to get rid of those infinite values, so we exclude them
        filtered_dict = {vertex: distance for vertex, distance in paths.items() if distance < 2147483647}
        
        # Now that we're sure there aren't infinity values, we sort the dict in descending order
        sorted_paths = sorted(filtered_dict.items(), key=lambda item: item[1], reverse=True)
        larg_paths = []
        amount_paths = len(sorted_paths)

        for i in range(10 if amount_paths>=10 else amount_paths):
            airport_code = sorted_paths[i][0]
            distance = sorted_paths[i][1]

            airport_info = self.adj_list[airport_code]

            larg_paths.append( (airport_info, distance) )

        # Finally return the larg_paths
        return larg_paths
    
    def reconstruct_path(self, airports: list):
        airports_code = airports.split(',')

        source = airports_code[0].strip()
        end = airports_code[1].strip()

        distance, pred = self.dijkstra(source)

        previous = end
        minimum_path = end

        while previous is not None:
            previous = pred[previous]

            if previous is not None:
                minimum_path += ' ' + previous
        else:
            minimum_path = minimum_path.split(' ')
            minimum_path.reverse()

        return minimum_path  

    def min_dist_not_visit(self, D, visit):
        min_dist, index = self.INF + 1, -1
        for code in self.adj_list:
            if D[code] < min_dist and not visit[code]:
                min_dist = D[code]
                index = code
        return index

    def all_visited(self, visit):
        for v in visit:
            if not visit[v]:
                return False
        return True

### Creating the graph

In [ ]:
graph = AirportGraph()

for index, row in df.iterrows():
    # Extract data from the DataFrame
    source_code = row['Source Airport Code']
    destination_code = row['Destination Airport Code']
    source_lat, source_long = row['Source Airport Latitude'], row['Source Airport Longitude']
    destination_lat, destination_log = row['Destination Airport Latitude'], row['Destination Airport Longitude'] 

    # Add airports if they don't already exist in the graph
    if source_code not in graph.adj_list:
        graph.add_vertex(source_code, row['Source Airport Name'], row['Source Airport City'],
            row['Source Airport Country'], source_lat, source_long
        )

    if destination_code not in graph.adj_list:
        graph.add_vertex(destination_code, row['Destination Airport Name'], row['Destination Airport City'],
            row['Destination Airport Country'], destination_lat, destination_log
        )

    # Add the flight
    graph.add_edge(source_code, destination_code,haversine(source_lat, source_long, destination_lat, destination_log))




### Map with all the data

In [ ]:

# Inicializar el mapa centrado en una ubicación arbitraria (puedes ajustar esto según tus necesidades)
map_center = [0, 0]
mymap = folium.Map(location=map_center, zoom_start=2)

# Agregar marcadores para cada vértice en el grafo
for airport_code, info in graph.adj_list.items():
    lat, lon = info['latitude'], info['longitude']
    name = info['name']
    city = info['city']
    country = info['country']

    # Crear un marcador en el mapa
    marker = folium.Marker(location=[lat, lon], 
                           popup=f"Code: {airport_code}<br>Name: {name}<br>{city}, {country}")

    # Agregar el marcador al mapa
    marker.add_to(mymap)

    # Agregar líneas representando aristas
    for destination, weight in info['flights']:
        dest_info = graph.adj_list[destination]
        dest_lat, dest_lon = dest_info['latitude'], dest_info['longitude']
        
        # Crear una línea entre los dos aeropuertos
        line = PolyLine(locations=[[lat, lon], [dest_lat, dest_lon]], color="blue")
        line.add_to(mymap)

# Guardar el mapa como un archivo HTML
mymap.save('airport_map.html')

#1: Given an airport, show the 10 largest paths...

In [ ]:
airport = 'BAQ'

In [ ]:
# We obtain the 10 largest airports 
largest_paths = graph.largest_paths(airport)
nodes = graph.adj_list

# Once we obtained these paths, let's create a map centered in the source node
map_center = [nodes[airport]['latitude'], nodes[airport]['longitude']]
mymap = folium.Map(location=map_center, zoom_start=4)

# The source airport will have a blue marker
orig_airport_info = nodes[airport]
tooltip_orig = f"<div style='width: 200px;'><b>{orig_airport_info['name']}</b> - <b>{orig_airport_info['airport_code']}</b><br></b>{orig_airport_info['city']}, {orig_airport_info['country']}<br><b>Latitud:</b> {orig_airport_info['latitude']}<br> <b>Longitud:</b> {orig_airport_info['longitude']}</div>"
folium.Marker(
    [orig_airport_info['latitude'], orig_airport_info['longitude']],
    popup=folium.Popup(tooltip_orig, max_width=300),
    icon=folium.Icon(color='blue')
).add_to(mymap)

# Here we add red markers to the other airports 
for path in largest_paths:
    airport_info = path[0]
    airport_name = airport_info['name']
    airport_code = airport_info['airport_code']
    airport_city = airport_info['city']
    airport_country = airport_info['country']
    airport_latitude = airport_info['latitude']
    airport_longitude = airport_info['longitude']
    distance = path[1]

    tooltip_info = f"<div style='width: 200px;'><b>{airport_name}</b> - <b>{airport_code}</b><br>{airport_city}, {airport_country}<br><b>Latitud:</b> {lat}<br><b>Longitud:</b> {lon}<br><b>Distancia:</b> {distance} km</div>"
    
    folium.Marker(
        [airport_latitude, airport_longitude],
        popup=folium.Popup(tooltip_info, max_width=300),
        icon=folium.Icon(color='red')
    ).add_to(mymap)

display(mymap)




#2: Given a second airport, display a map with the shortest path between...

In [ ]:
airports = 'COK, GIG'

In [ ]:
# First, we obtain the minimum path
minimum_path = graph.reconstruct_path(airports)

# Again, we create a map centered in the first airport 
source_node = minimum_path[0]
nodes = graph.adj_list

map_center = [nodes[source_node]['latitude'], nodes[source_node]['longitude']] 
mymap = folium.Map(location=map_center, zoom_start=2)

# Adding markers to all the airports in the map
for airport_code in minimum_path:
    node = nodes[airport_code]
    lat, lon = node['latitude'], node['longitude']
    name = node['name']
    city = node['city']
    country = node['country']

    popup_text = f"<div style='width: 200px;'><b>{name}</b> - <b>{airport_code}</b><br>{city}, {country}<br><b>Latitud:</b> {lat}<br><b>Longitud:</b> {lon}</div>"
    popup = folium.Popup(popup_text, max_width=300)
    marker = folium.Marker(location=[lat, lon], popup=popup)
    
    # Adding the marker to the map
    marker.add_to(mymap)

# To represent the minimum path, we draw a line between the nodes
for i in range(len(minimum_path) - 1):
    current_info = nodes[minimum_path[i]]
    next_info = nodes[ minimum_path[i + 1] ]

    line = folium.PolyLine(locations=[[current_info['latitude'], current_info['longitude']],
                                      [next_info['latitude'], next_info['longitude']]], color="red")
    line.add_to(mymap)

display(mymap)



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=281142bb-5c31-48ab-971c-9b63a8e075a9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>